<a href="https://colab.research.google.com/github/moamanuldin/my_track/blob/main/billing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Latest update allows us to safe the data to a transaction file and clear the cart at checkout

In [ ]:
products = {
    "Apple": {"price": 2.5, "quantity": 3},
    "Banana": {"price": 1.0, "quantity": 5}
}

total = 0
for key, details in products.items():
  if details["quantity"] * details["price"] > 5:
    total += details["price"] * details["quantity"] * 0.9
  else:
    total += details["price"] * details["quantity"]
print(total)

11.75


In [40]:
import pandas as pd
import datetime

In [33]:
stock = {
    "Apple": {"Stock": 5, "Price": 20},
    "Banana": {"Stock": 12, "Price": 17},
    "Mango": {"Stock": 10, "Price": 25}
}
cart = {}
# Convert the stock dictionary to a pandas DataFrame
stock_df = pd.DataFrame.from_dict(stock, orient='index')
# Save the DataFrame to a CSV file
stock_df.to_csv('stock_data.csv')

print("Stock data saved to a csv file")

Stock data saved to a csv file


In [22]:
print(stock)

{'Apple': {'Stock': 4, 'Price': 20}, 'Banana': {'Stock': 12, 'Price': 17}, 'Mango': {'Stock': 10, 'Price': 25}}


In [39]:
TAX_RATE = 1.12
transactions = []
def show_items():
    print("Available items:")
    for name, info in stock.items():
        print(f"- {name}: {info['Stock']} in stock at ${info['Price']} each")
    print()


def get_valid_choice():
    while True:
        purchase = input("Choose an item: ").strip().title()
        if purchase in stock:
            if stock[purchase]["Stock"] > 0:
                return purchase
            else:
                print(f"Sorry, {purchase} is out of stock.")
        else:
            print("Item not found. Please choose from the list.")


def get_valid_amount(purchase):
    max_amount = stock[purchase]["Stock"]
    while True:
        amount_str = input(f"Enter quantity (1–{max_amount}): ").strip()
        if amount_str.isdigit():
            amount = int(amount_str)
            if 1 <= amount <= max_amount:
                return amount
        print("Invalid quantity; please try again.")


def calculate_subtotal():
    subtotal = sum(stock[item]["Price"] * data["amount"] for item, data in cart.items())
    return round(subtotal * TAX_RATE, 2)


def handle_new_item(purchase):
    amount = get_valid_amount(purchase)
    cart[purchase] = {
        "amount": amount,
        "price": stock[purchase]["Price"],
        "total": amount * stock[purchase]["Price"]
        }
    stock[purchase]["Stock"] -= amount
    print(f"Added {amount} x {purchase} to cart.")


def handle_existing_item(purchase):
    current = cart[purchase]["amount"]
    print(f"You have {current} x {purchase} in your cart.")
    while True:
        action = input("Choose: (1) Adjust quantity, (2) Continue shopping, (3) Checkout: ").strip()
        if action == '1':
            new_amount = get_valid_amount(purchase)
            diff = new_amount - current
            stock[purchase]["Stock"] -= diff
            cart[purchase]["amount"] = new_amount
            print(f"Updated {purchase} to {new_amount} units in cart.")
            return 'continue'
        elif action == '2':
            return 'continue'
        elif action == '3':
            return 'checkout'
        else:
            print("Invalid option; please enter 1, 2, or 3.")

def save_transaction(cart):
  items = [(product_name, details["amount"]) for product_name, details in cart.items()]

  new_transaction = [
      {
          "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
          "items": items,
          "total": calculate_subtotal()
      }
                     ]
  transactions_df = pd.DataFrame(new_transaction)
  transactions_df.to_csv(
    'transactions.csv',
    mode='a',
    header=False,
    index=False
    )


def checkout():
    total = calculate_subtotal()
    print("\nCheckout Summary:")
    for purchase, data in cart.items():
        price = stock[purchase]["Price"]
        print(f"- {purchase}: {data['amount']} x ${price} = ${price * data['amount']}")
    print(f"Total with tax: ${total}\n")
    confirm = input("Proceed to payment? (y/n): ").strip().lower()
    if confirm == 'y':
        print("Payment successful. Thank you for shopping!")
        # cart[purchase]["time"] = datetime.datetime.now()
        # save_transaction(cart)
        return True
    else:
        print("Checkout canceled; returning to shopping.")
        return False


# Save transaction to a csv file
def to_csv(cart):
  cart_df = pd.DataFrame.from_dict(cart, orient='index')
  cart_df.to_csv('cart_data.csv')

def main():
    print("Welcome to the shop!\n")
    while True:
        show_items()
        purchase = get_valid_choice()
        if purchase in cart:
            action = handle_existing_item(purchase)
        else:
            handle_new_item(purchase)
            action = input("Continue shopping or checkout? (c/checkout): ").strip().lower()
        if action in ('checkout', '3'):
            if checkout():
                # to_csv(cart)
                save_transaction(cart)
                cart.clear()
                break


if __name__ == '__main__':
    main()


Welcome to the shop!

Available items:
- Apple: 3 in stock at $20 each
- Banana: 10 in stock at $17 each
- Mango: 10 in stock at $25 each

Choose an item: apple
Enter quantity (1–3): 1
Added 1 x Apple to cart.
Continue shopping or checkout? (c/checkout): checkout

Checkout Summary:
- Apple: 1 x $20 = $20
Total with tax: $22.4

Proceed to payment? (y/n): y
Payment successful. Thank you for shopping!
